In [1]:
import os 

# Run global_variables, synthesize_functions and mlp_model
%run ~/violin-renderer/src/global_variables.ipynb
%run ~/violin-renderer/src/models/lstm/lstm_model.ipynb
%run ~/violin-renderer/src/synthesize/synthesize_functions.ipynb
%run ~/violin-renderer/src/models/lstm/data_processing.ipynb

Start downloading MuseScore General soundfont.
MuseScore General soundfont has successfully been downloaded to : /home/wanninglu/.muspy/musescore-general.


In [2]:
testing_X, testing_y = load_testing_data()

In [3]:
def trim_sequence(input_song, true_song):
    all_source_inputs = []
    all_ground_truths = []

    for note_source, note_truth in zip(input_song, true_song):
        if note_source[1] > 64 * 24:
            break
            
        note_source = list(note_source)
        note_truth = list(note_truth)
        
        note_source[0] = scale_timings(note_source[0])
        note_source[1] = scale_timings(note_source[1])
        note_source[2] = scale_pitch(note_source[2])
        
        all_source_inputs.append(note_source)
        all_ground_truths.append(note_truth)

    return all_source_inputs, all_ground_truths

In [4]:
# Create an output directory in /src, generate all results from the test dataset and synthesize all of them
# Setting up paths and output folders
SOURCE_INPUT_PATH =  HOME_PATH + '/bach-violin-dataset/dataset/source-input'
TEMPO_PATH =  HOME_PATH + '/bach-violin-dataset/dataset/tempos'
OUTPUT_PATH = HOME_PATH + '/src/mlp-output'
composer_directories = os.listdir(SOURCE_INPUT_PATH)

In [5]:
input_dim = 3
hidden_dim = 32
layer_dim = 2
output_dim = 2

# initialize the MLP
model = LSTM(input_dim, hidden_dim, layer_dim, output_dim)

model.load_state_dict(torch.load(HOME_PATH + '/src/models/lstm/lstm.pt'))

<All keys matched successfully>

In [27]:
# Get generated output values from all songs
results = {}

for input_path, true_path in zip(testing_X, testing_y):
    trimmed_notes = []
    trimmed_notes, trimmed_truth = trim_sequence(testing_X[input_path], testing_y[true_path])
    trimmed_notes = [trimmed_notes]
    trimmed_notes = torch.Tensor(trimmed_notes)
    length = [len(trimmed_notes[0])]
    length = torch.Tensor(length)

    pred = model(trimmed_notes, length)
    pred = pred.tolist()
    results[input_path] = pred

print(results)

{'/home/wanninglu/violin-renderer/bach-violin-dataset/dataset/source-input/shunske-sato/bwv1005/shunske-sato_bwv1005_mov2.csv': [[[3.141122817993164, 2.9310998916625977], [3.407043218612671, 3.1995317935943604], [3.4949913024902344, 3.283342123031616], [3.526832103729248, 3.3100178241729736], [3.5805304050445557, 3.3605332374572754], [3.6933460235595703, 3.4732351303100586], [3.876474380493164, 3.6593921184539795], [4.146888732910156, 3.935739755630493], [4.494876861572266, 4.291881561279297], [4.908596038818359, 4.715633392333984], [5.379864692687988, 5.198590278625488], [5.894609451293945, 5.726419448852539], [6.433956146240234, 6.280050277709961], [6.988604545593262, 6.850028991699219], [7.549668312072754, 7.427135467529297], [8.10299015045166, 7.996835708618164], [8.633086204528809, 8.543268203735352], [9.127821922302246, 9.053925514221191], [9.576661109924316, 9.517807960510254], [9.968637466430664, 9.923585891723633], [10.330670356750488, 10.298158645629883], [10.663670539855957,

In [28]:
# Synthesize
if not os.path.isdir(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

for notes_path, generated_results in results.items():
    name = '/' + notes_path.split('/')[-1]
    name = name.replace('.csv', '')
    generated_timings = [(note[0], note[1]) for note in generated_results]
    synthesize_generated_output(notes_path, generated_timings, OUTPUT_PATH, name)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()